In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_score
import metrics
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from progressbar import ProgressBar
import data_grab
from time import time
from sklearn.externals import joblib
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_selection import VarianceThreshold



In [2]:
def contest_metric(numpy_array_predictions, numpy_array_actual_values):
    return metrics.weighted_rmsle(numpy_array_predictions, numpy_array_actual_values,
            weights=metrics.KEEPING_IT_CLEAN_WEIGHTS)

In [3]:
def contest_scoring(X, y, pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    s1 = pipeline.fit(X_train, y_train['score_lvl_1']).predict(X_test)
    s2 = pipeline.fit(X_train, y_train['score_lvl_2']).predict(X_test)
    s3 = pipeline.fit(X_train, y_train['score_lvl_3']).predict(X_test)
    results = np.dstack((s1, s2, s3))
    score = contest_metric(np.round(results[0]), np.array(y_test))
    print("Contest score of {}".format(score))
    return score

In [4]:
from sklearn.metrics import accuracy_score

def raw_scoring(p1, p2, p3, ytrue):
    '''since cross_val_score doesn't allow you to round the results beforehand. also for pymc3 and other non-sklearn models'''
    score1 = accuracy_score(ytrue['score_lvl_1'], np.clip(np.round(p1), 0, np.inf))
    print("Level 1 accuracy score of {}".format(score1))
    score2 = accuracy_score(ytrue['score_lvl_2'],np.clip(np.round(p2), 0, np.inf))
    print("Level 2 accuracy score of {}".format(score2))
    score3 = accuracy_score(ytrue['score_lvl_3'], np.clip(np.round(p3), 0, np.inf))
    print("Level 3 accuracy score of {}".format(score3))
    
    results = np.dstack((p1, p2, p3))[0]
    rounded = np.clip(np.round(results), 0, np.inf)
    score = contest_metric(rounded, np.array(ytrue))
    print("Contest score of {}".format(score))
    
    compare = pd.concat([pd.DataFrame(np.concatenate((results, rounded), axis=1)), ytrue.reset_index(drop=True)], axis=1)
    compare.columns = ['pred1','pred2','pred3','round1','round2','round3','true1','true2','true3']
    compare['offset1'] = compare.round1-compare.true1
    compare['offset2'] = compare.round2-compare.true2
    compare['offset3'] = compare.round3-compare.true3
        
    return score1, score2, score3, score, compare.head(10)

    
def raw_fit(X, y, pipeline):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=42)
    
    p1 = pipeline.fit(xtrain, ytrain['score_lvl_1']).predict(xtest)
    p2 = pipeline.fit(xtrain, ytrain['score_lvl_2']).predict(xtest)
    p3 = pipeline.fit(xtrain, ytrain['score_lvl_3']).predict(xtest)
        
    return p1, p2, p3, ytest

In [5]:
def extract_features(df):
    features = df.drop(['score_lvl_1', 'score_lvl_2', 'score_lvl_3'], axis=1)
    response = df[['score_lvl_1', 'score_lvl_2', 'score_lvl_3']].astype(np.int8)
    
    return features, response

In [6]:
combo = pd.read_pickle('pickle_jar/pre-pivot_all_review_combo_365')
df = pd.read_pickle("pickle_jar/non_review_df_365")

In [160]:
df = data_grab.get_selects('train')

In [161]:
# not getting rid of everything. not getting rid of ancient reviews if they correspond with an ancient inspection.
# just getting rid of ancient reviews in relation to a specific inspection
df = df[df.review_delta <= 365]

In [162]:
# making a column that enumerates the sorted review delta.
# removes any nans (usually marked as such because the reviews happend after an inspection date) beforehand.
# this can completely remove a restaurant if it has no reviews left after nan drop. might add back in later
df = df.sort('review_delta')
df2 = df.dropna(subset=['review_delta']).groupby('inspection_id').cumcount()

In [163]:
# df3 = pd.concat([df.dropna(subset=['review_delta']), df2], axis=1)
df = pd.concat([df.dropna(subset=['review_delta']), df2], axis=1)

In [164]:
df = df.rename(columns={0:'enumerated_review_delta'})
del df2

In [165]:
# add back in the restaurants that were completely removed because none of their reviews were
# not a significant loss in the number of restaurants. going to just leave it out for now
# mask = list(set(df.restaurant_id) - set(df3.restaurant_id))
# pd.concat([df[df.restaurant_id.isin(mask)], df3])

In [166]:
df.shape

(500936, 137)

In [167]:
just_review_data = df[[ 
 'inspection_id',
 'enumerated_review_delta',
#  'review_id',
 'review_delta', 
#  'review_date', 
 'review_stars',
 'review_text',
#  'user_id',
 'review_votes_cool',
 'review_votes_funny',
 'review_votes_useful',
 'user_average_stars',
#  'user_fans',
 'user_review_count',
#  'user_votes_cool',
#  'user_votes_funny',
#  'user_votes_useful',
 'user_yelping_since_delta',
#  'review_year',
#  'review_month',
#  'review_day',
#  'review_dayofweek',
#  'review_quarter',
#  'review_dayofyear',
 'user_ever_elite',
 ]]

In [168]:
# this removes all the nans left. review stars as nans because reviews and tips combined and tips didnt give a star rating
# planning to treat this as a categorical variable later so making it zero shouldnt affect anything
just_review_data.review_stars = just_review_data.review_stars.fillna(0)

In [28]:
just_review_data.to_pickle('pickle_jar/pre-pivot_365')

In [127]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
lbl_enc = LabelEncoder()
# df['restaurant_id_enc'] = lbl_enc.fit_transform(df.restaurant_id)

In [122]:
sentiment = pd.read_pickle('pickle_jar/review_text_sentiment_pivot')
tfidf = joblib.load('pickle_jar/tfidf_preprocessed_ngram3_sublinear_1mil_pivot')
similarity = pd.read_pickle('pickle_jar/similarity_vectors_pivot')

In [132]:
sentiment = sentiment[['inspection_id', 'enumerated_review_delta', 'polarity', 'subjectivity', 'neg', 'neu', 'pos', 'compound']]

In [137]:
similarity.drop(['review_delta', 'review_stars', 'review_text', 'review_votes_cool', 'review_votes_funny',
                 'review_votes_useful', 'user_average_stars', 'user_fans', 'user_review_count', 'user_votes_cool', 
                 'user_votes_funny', 'user_votes_useful', 'user_yelping_since_delta', 'user_ever_elite',
                 'preprocessed_review_text'], axis=1, inplace=True)

In [159]:
combo = pd.merge(sentiment, similarity, on=['inspection_id', 'enumerated_review_delta'])

In [170]:
just_review_data.drop('review_text', axis=1, inplace=True)

/Users/amangum/anaconda/envs/datasci/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [173]:
combo = pd.merge(combo, just_review_data, on=['inspection_id', 'enumerated_review_delta'])

In [174]:
combo.to_pickle('pickle_jar/pre-pivot_all_review_combo_365')

In [198]:
df.user_ever_elite.value_counts(dropna=False)

False    334845
True     166091
dtype: int64

In [199]:
df['review_stars'] = df.review_stars.fillna(0).astype('category')
df.restaurant_attributes_price_range = df.restaurant_attributes_price_range.fillna(df.restaurant_attributes_price_range.median())

In [206]:
df.drop(['review_text', 'review_date', 'user_id', 'restaurant_full_address', 'restaurant_name',
         'inspection_date', 'inspection_id', 'inspection_date',  
         'restaurant_attributes_by_appointment_only', 'restaurant_open',
        'review_year',
 'review_month',
 'review_day',
 'review_dayofweek',
 'review_quarter',
 'review_dayofyear',
 'inspection_dayofyear'], axis=1, inplace=True)

df.drop(['restaurant_neighborhoods', 'restaurant_categories'], axis=1, inplace=True)

In [207]:
df.to_pickle("pickle_jar/non_review_df_365")

#combine the user average stars and user review count etc into a trustworthiness variable

In [176]:
combo.sort(['inspection_id', 'enumerated_review_delta']).set_index(['inspection_id', 'enumerated_review_delta']).T

inspection_id                                                         0      \
enumerated_review_delta                                                  0    
polarity                                                          0.2229464   
subjectivity                                                      0.7133929   
neg                                                                       0   
neu                                                                   0.783   
pos                                                                   0.217   
compound                                                             0.9636   
manager                   [0.116296313703, 0.116296313703, 0.09853885322...   
supervisor                [0.180711179972, 0.128739342093, 0.12695443630...   
training                  [0.328027784824, 0.157236814499, 0.15316183865...   
safety                    [0.358425438404, 0.313093900681, 0.22312483191...   
disease                   [0.178598597646, 0.174537301064, 0.15921005606...   
ill                       [0.224779933691, 0.19681827724, 0.190300583839...   
sick                      [0.266563624144, 0.258543759584, 0.25563034415...   
poisoning                 [0.266977906227, 0.154491618276, 0.12765660882...   
hygiene                   [0.403521507978, 0.240984976292, 0.16647472977...   
raw                       [0.339037925005, 0.318922072649, 0.30474078655...   
undercooked               [0.396595746279, 0.37989500165, 0.299409240484...   
cold                      [0.409688562155, 0.409688562155, 0.22290925681...   
clean                     [0.261485755444, 0.257004648447, 0.24404911696...   
sanitary                  [0.371469497681, 0.242693945765, 0.16444574296...   
wash                      [0.325484097004, 0.325484097004, 0.24262194335...   
jaundice                  [0.172405004501, 0.141730055213, 0.14173005521...   
yellow                    [0.286812782288, 0.228999257088, 0.22899925708...   
hazard                    [0.352810382843, 0.279726475477, 0.19018650054...   
inspection                [0.183560550213, 0.168573096395, 0.16153559088...   
violation                 [0.204849630594, 0.12797345221, 0.12347856164,...   
gloves                    [0.289221763611, 0.289221763611, 0.22099322080...   
hairnet                   [0.289698749781, 0.288964867592, 0.26448917388...   
nails                     [0.194717153907, 0.194717153907, 0.18863959610...   
jewelry                   [0.234194412827, 0.149923667312, 0.14378458261...   
...                                                                     ...   
contaminated              [0.278205811977, 0.259442746639, 0.24917797744...   
gross                     [0.154287859797, 0.141207844019, 0.13841477036...   
disgusting                [0.327441036701, 0.308370053768, 0.29797932505...   
stink                     [0.289710044861, 0.268125891685, 0.25012582540...   
old                       [0.16796554625, 0.164793789387, 0.155218586326...   
parasite                  [0.173846319318, 0.142110988498, 0.12603498995...   
reheat                    [0.356566250324, 0.308967858553, 0.29918843507...   
frozen                    [0.368782520294, 0.341188162565, 0.34118816256...   
broken                    [0.207048565149, 0.202479183674, 0.20110626518...   
drip                      [0.298404991627, 0.298404991627, 0.22234307229...   
bathroom                  [0.242184981704, 0.230421304703, 0.18717461824...   
toilet                    [0.215910956264, 0.175648808479, 0.17181999981...   
leak                      [0.157449916005, 0.140461146832, 0.13238364458...   
trash                     [0.212857708335, 0.19239833951, 0.172082290053...   
dark                      [0.213061317801, 0.213061317801, 0.19345648586...   
lights                    [0.17151388526, 0.164933905005, 0.158460006118...   
dust                      [0.247481316328, 0.247481316328, 0.22925849258...   
puddle                    [0.323749572039, 0.32374

In [9]:
# combo.pivot(index='inspection_id', columns='enumerated_review_delta')
combo.set_index(['inspection_id', 'enumerated_review_delta']).unstack(1)
# combo.pivot_table(index='inspection_id', columns='enumerated_review_delta')

polarity                                          \
enumerated_review_delta       0         1         2         3         4     
inspection_id                                                               
0                        0.222946  0.237602       NaN       NaN       NaN   
1                        0.189981  0.115152  0.324259       NaN       NaN   
2                        0.005025 -0.234226  0.700000  0.158102  0.019551   
7                        0.193750       NaN       NaN       NaN       NaN   
8                        0.214286 -0.059524  0.166833  0.030000  0.259574   
9                       -0.225000 -0.156003  0.171605       NaN       NaN   
10                      -0.300000  0.021298  0.164500  0.069048  0.247436   
12                       0.070000  0.016346 -0.222222  0.600000       NaN   
13                       0.196212  0.156618  0.187500  0.422240  0.268182   
14                      -0.059730 -0.283333 -0.208333  0.125757 -0.109040   
15                       0.033333 -0.225556  0.325694 -0.036000  0.112923   
16                       0.533333       NaN       NaN       NaN       NaN   
17                       0.034028  0.294792  0.198214 -0.005556 -0.113500   
18                       0.300000  0.202500  0.370000  0.519048  0.328114   
19                       0.000000  0.417308  0.192949  0.036667  0.320312   
23                       0.131349 -0.103000  0.198947  0.196883  0.343750   
24                       0.450000  0.287292  1.000000  0.098106  0.000000   
27                       0.137415  0.300735  0.135119  0.387500  0.115625   
28                       0.152106  0.102381 -0.450000  0.341165  0.236556   
29                       0.218202  0.212468  0.296484  0.310900  0.157185   
30                       0.104861  0.147377       NaN       NaN       NaN   
32                       0.244273  0.190606  0.800000  0.416667  0.082930   
33                       0.083333  0.101017  0.158102  0.116667  0.340000   
34                       0.206389  0.283333  0.018056  0.131371  0.077425   
36                       0.183333 -0.044097       NaN       NaN       NaN   
37                       0.643333  0.116017  0.211640 -0.380000  0.447917   
38                       0.800000  0.090646  0.260937  0.230303       NaN   
40                       0.353802  0.183333  0.202841  0.150641  0.575000   
42                      -0.068750  0.159470  0.109664  0.262235  0.508333   
43                      -0.015119  0.000000 -0.206746  0.098958  0.119271   
...                           ...       ...       ...       ...       ...   
33819                    0.000000  0.625000  0.219786  0.000000 -0.098073   
33820                    0.181000 -0.278704  0.250000  0.269038  0.432500   
33821                    0.090000  0.375000 -0.135779  0.000000 -0.500000   
33822                    0.644271  0.166760  0.625000  0.000000  0.230511   
33824                    0.262500  0.316964  0.375465  0.015404  0.485268   
33825                    0.307353  0.406944       NaN       NaN       NaN   
33826                    0.104040  0.475000  0.003139  0.121429       NaN   
33827                    0.350000       NaN       NaN       NaN       NaN   
33828                    0.223148  0.166667  0.115000  0.131944  0.488889   
33829                    0.471429  0.230556  0.183333       NaN       NaN   
33830                   -0.453333  0.054565  0.136905  0.039062 -0.325758   
33831                    0.141667 -0.050000 -0.042308  0.500000  0.255602   
33834                    0.113194  0.129583  0.273611  0.356548       NaN   
33835                    0.062847  0.036813  0.400000  0.065476  0.174167   
33836                    0.347222  0.050000 -0.015278       NaN       NaN   
33837                   -0.234524 -0.247976  0.309425  0.067708  0.000000   
33838                    0.023203  0.001683  0.014660  0.155268  0.117710   
33839                   -0.409375  0.276786  0.058333       NaN       NaN   
33842                

In [186]:
X, y = extract_features(pd.merge(combo, df[['inspection_id', 'enumerated_review_delta', 'score_lvl_1', 'score_lvl_2', 'score_lvl_3']], on=['inspection_id', 'enumerated_review_delta']))

In [10]:
df

,restaurant_id,review_id,review_stars,review_votes_cool,review_votes_funny,review_votes_useful,user_average_stars,user_compliments_cool,user_compliments_cute,user_compliments_funny,...,restaurant_category_6,restaurant_category_7,review_delta,previous_inspection_delta,inspection_year,inspection_month,inspection_day,inspection_dayofweek,inspection_quarter,enumerated_review_delta
3673133,njoZyY3r,jlKVXCHaSrj-cgYBa-bshQ,3,0,0,0,3.27,NaN,NaN,NaN,...,NaN,NaN,0,168,2014,6,23,0,2,0
3416909,eVOBNMOj,FFJmcIuQyG5EW9f4MvLC6w,4,0,0,1,3.68,8,NaN,6,...,Restaurants,NaN,0,115,2010,4,15,3,2,0
2640320,WwOaXpoB,miL2NBTI2P5dRQoTci0xgg,5,0,0,1,3.76,2,1,NaN,...,NaN,NaN,0,14,2012,4,10,1,2,0
3415000,eVOBNMOj,8ehwidyYdEwLjh0Q9iJnvg,3,0,0,0,2.86,NaN,NaN,NaN,...,Restaurants,NaN,0,35,2011,9,30,4,3,0
2641188,0ZEDQG3D,__nql_JwgOdCOoVCvGHOJg,3,0,0,0,3.86,17,1,1,...,NaN,NaN,0,8,2014,1,7,1,1,0
2641304,0ZEDQG3D,NHCQTcV0kOCHqviPkjzUhQ,4,2,0,0,3.66,4,NaN,1,...,NaN,NaN,0,169,2012,7,6,4,3,0
1720355,lnORKGEN,HupcUbv2BP7f_SSQy1gaAw,4,0,0,0,3.68,2,NaN,6,...,NaN,NaN,0,176,2015,1,30,4,1,0
3769370,V430Lq3B,yy6bepLQ-nCR-Tdh1ucodw,5,0,0,0,4.50,NaN,NaN,NaN,...,NaN,NaN,0,227,2013,8,14,2,3,0
3769371,V430Lq3B,rpi00iBTaLxMj7TCG9q2qw,3,0,0,0,3.79,NaN,NaN,NaN,...,NaN,NaN,0,227,2013,8,14,2,3,1
2219340,dj3dqNo9,1rqAvSTcpNV5uf-MTBDczg,4,0,0,0,3.76,20,NaN,20,...,NaN,NaN,0,83,2012,3,22,3,1,0


In [189]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor 
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
import sendMessage


from sklearn.preprocessing import MinMaxScaler

t0 = time()


# set classifiers to test
estimator = LinearRegression()
# estimator = RandomForestClassifier(n_jobs=-1, random_state=42)
# estimator = SGDClassifier(n_jobs=-1, random_state=42)
# estimator = Perceptron(n_jobs=-1, random_state=42)  # gets some nuances
# estimator = SGDRegressor() # gets some nuances
# estimator = KNeighborsClassifier()
# estimator = KNeighborsRegressor()  # gets some nuances
# estimator = DecisionTreeClassifier()
# estimator = DecisionTreeRegressor()
# estimator = GaussianNB()
# estimator = MultinomialNB()
# estimator = LinearSVC(random_state=42)

pipeline = Pipeline([
#         ('zero_variance_removal', VarianceThreshold()),
#         ('k_best', SelectKBest(score_func=f_classif, k=20)),
#         ('no_negative', MinMaxScaler()),
#         ('normalizer', Normalizer()),
#         ('normalizer', Normalizer(norm='l2')), #  for text classification and clustering
#         ('normalizer', Normalizer(copy=False)),
#         ('scaler', StandardScaler()),
#         ('scaler', StandardScaler(with_mean=False)), #  for sparse matrix
        ('clf', estimator),
])

p1,p2,p3,ytest = raw_fit(X, y, pipeline)
raw_scoring(p1,p2,p3,ytest)


print("{} seconds elapsed".format(time()-t0))

sendMessage.doneTextSend(t0, time(), 'model test')




ValueError: setting an array element with a sequence.